# Shadowing Envelope

## 0. Initialization

importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np


# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

## 1. Import Meshes

### 1.1 Load Meshes

In [2]:
envelope_path = os.path.relpath('../data/meshes/compulsory_envelope.obj')
context_path = os.path.relpath('../data/meshes/immediate_context.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

True
False


### 1.2 Visualize Meshes 

In [3]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
# p.show()

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001B6989ACF40) at 0x000001B6A333E7C0>

## 2. Import Lattice 

### 2.1. Load both the Envelope and Shadow Lattice

In [4]:
# loading the lattice from csv
lattice = os.path.relpath('../data/meshes/voxelized_envelope.csv')
envelope_lattice = tg.lattice_from_csv(lattice)


### 2.2. Visualize the Envelope Lattice

In [5]:
# initiating the plotter
p = pv.Plotter()

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
# p.show()

<vtkmodules.vtkRenderingOpenGL2.vtkOpenGLActor(0x000001B6989A9B40) at 0x000001B6997D15E0>

### 2.3 Store shadowing information in a Lattice

In [16]:
# Loading the shadow lattice from csv
shadow_lattice = os.path.relpath('../data/fields/shadowing.csv')
shadow_envelope_lattice = tg.lattice_from_csv(shadow_lattice)

### 2.4. Visualize the shadowing

In [17]:
# initiating the plotter
p = pv.Plotter(notebook=True)

l = shadow_envelope_lattice
# remapping
l = 250 * (l - l.min()) / l.max()

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = l.shape
# The bottom left corner of the data set
grid.origin = l.minbound
# These are the cell sizes along each axis
grid.spacing = l.unit

# Add the data values to the cell data
grid.point_arrays["Shadowing"] = l.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.5, 1.0],opacity=opacity, shade=True)

# plotting
# p.show()

<vtkmodules.vtkRenderingCore.vtkVolume(0x000001B69FACAC80) at 0x000001B6AF1B2CA0>

## 3. Remove Result in shadow Lattice

### 3.0. Interpolating the shadowing result into higher resolution

In [9]:
# from scipy.interpolate import RegularGridInterpolator
# import functions

# lattice_path_highres = os.path.relpath('../data/meshes/voxelized_envelope_highres.csv')
# envelope_lattice_highres = tg.lattice_from_csv(lattice_path_highres)

# shadow_envelope_lattice_highres = functions.interpolate(shadow_envelope_lattice, envelope_lattice_highres)
# envelope_lattice_highres.shape

(31, 18, 8)

### 3.1. Defining maximum shadowing

In [18]:
# defining the shadow envelope lattice with values calcultated in shadowing
shadow_envelope_lattice_highres

# shadowing treshold
shadowing_treshold = 0.3 # number is subject to change

# removing values above set treshold and setting them to zero
new_envelope_lattice = np.where(shadow_envelope_lattice_highres > shadowing_treshold,0,shadow_envelope_lattice_highres)

# set all usable values to 1

usable_lattice = np.where(new_envelope_lattice > 0, 1,new_envelope_lattice)
usable_lattice = usable_lattice.astype(bool)
usable_lattice = tg.to_lattice(usable_lattice, shadow_envelope_lattice_highres)
usable_lattice.shape

(31, 18, 8)

### 3.2. Visualization of leftover voxels

In [19]:
# initiating the plotter
p = pv.Plotter()

# fast visualization of the lattice
usable_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.8088446466429, 708.392439646643, 742.1697567966429),
 (65.08283250000001, -12.333572500000002, 21.44374465),
 (0.0, 0.0, 1.0)]

## 4. Finalize Lattice work and save to csv

### 4.1. Save Usable Lattice inta a CSV 

In [20]:
# # save the sun access latice to csv

csv_path = os.path.relpath('../data/meshes/useable_lattice.csv')
usable_lattice.to_csv(csv_path)